In [2]:
cd "C:\Users\NQ10058846\PycharmProjects\Obesity disease risk prediction"

C:\Users\NQ10058846\PycharmProjects\Obesity disease risk prediction


# set the imports 

In [15]:
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from lightgbm import LGBMClassifier
from lib.utilities.helper_functions import HelperFunctions

In [11]:
train_data = pd.read_csv('datasets/train.csv')



In [14]:
train_data.head()

,id,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
0,0,Male,24.443011,1.699998,81.669950,yes,yes,2.000000,2.983297,Sometimes,no,2.763573,no,0.000000,0.976473,Sometimes,Public_Transportation,6
1,1,Female,18.000000,1.560000,57.000000,yes,yes,2.000000,3.000000,Frequently,no,2.000000,no,1.000000,1.000000,no,Automobile,1
2,2,Female,18.000000,1.711460,50.165754,yes,yes,1.880534,1.411685,Sometimes,no,1.910378,no,0.866045,1.673584,no,Public_Transportation,0
3,3,Female,20.952737,1.710730,131.274851,yes,yes,3.000000,3.000000,Sometimes,no,1.674061,no,1.467863,0.780199,Sometimes,Public_Transportation,4
4,4,Male,31.641081,1.914186,93.798055,yes,yes,2.679664,1.971472,Sometimes,no,1.979848,no,1.967973,0.931721,Sometimes,Public_Transportation,6


# perform explorative data analysis

In [27]:
"""
    HelperFunctions().generate_profile_report(train_data ,"ydata_profiling_settings_folder/ydata_profiling_settings.yaml" , "exploratory_data_analysis_output_folder/data_explorative_analysis_report")
"""

'\n    HelperFunctions().generate_profile_report(train_data ,"ydata_profiling_settings_folder/ydata_profiling_settings.yaml" , "exploratory_data_analysis_output_folder/data_explorative_analysis_report")\n'

# train and tune the predictive model 

In [12]:
# Encode target variable into numerical labels
label_encoder = LabelEncoder()
train_data['NObeyesdad'] = label_encoder.fit_transform(train_data['NObeyesdad'])


In [45]:
# Split features and target variable

X = train_data.drop(columns=['NObeyesdad'])
y = train_data['NObeyesdad']

In [46]:
# Define categorical and numerical features

categorical_features = ['Gender', 'family_history_with_overweight', 'FAVC', 'CAEC', 'SMOKE', 'SCC', 'CALC', 'MTRANS']
numerical_features = ['Age', 'Height', 'Weight', 'FCVC', 'NCP', 'CH2O', 'FAF', 'TUE']

In [47]:
# Split the data into training and validation sets

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [48]:
# Define preprocessing steps
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ])

In [49]:
# Define the LGBM model
lgbm_model = LGBMClassifier()

In [50]:
# Define the pipeline
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('classifier', lgbm_model)])

In [51]:
# Define hyperparameters for randomized search
param_grid = {
    'classifier__n_estimators': [100, 200, 300, 400, 500],
    'classifier__max_depth': [3, 5, 7, 9, 11],
    'classifier__learning_rate': [0.01, 0.05, 0.1, 0.2, 0.3],
    'classifier__subsample': [0.5, 0.7, 0.9, 1.0],
    'classifier__colsample_bytree': [0.5, 0.7, 0.9, 1.0],
    'classifier__min_child_samples': [10, 20, 30, 40, 50],
    'classifier__reg_alpha': [0.0, 0.1, 0.5, 1.0],
    'classifier__reg_lambda': [0.0, 0.1, 0.5, 1.0],
    'classifier__min_child_weight': [1e-3, 1e-2, 0.1, 1, 10]
}

In [ ]:
# Perform randomized search cross-validation

random_search = RandomizedSearchCV(pipeline, param_distributions=param_grid, 
                                   n_iter=50, 
                                   cv=3, 
                                   verbose=2, 
                                   random_state=42, 
                                   n_jobs=-1)
random_search.fit(X_train, y_train)

Fitting 3 folds for each of 50 candidates, totalling 150 fits


In [ ]:
# Get the best parameters

best_params = random_search.best_params_
best_model = random_search.best_estimator_

In [ ]:
# Print the best parameters
print("Best Parameters:", best_params)

In [18]:
# Evaluate the best model on the validation set
accuracy = best_model.score(X_test, y_test)
print("Validation Accuracy:", accuracy)

NameError: name 'X_test' is not defined

In [ ]:
# best model on entire dataset
best_model.fit(X,y)

In [19]:
# Prepare test data

test_data = pd.read_csv('datasets/test.csv')

In [20]:
# Make predictions on test data
test_predictions = best_model.predict(test_data)

In [21]:
# Create submission dataframe
submission_df = pd.DataFrame({'id': test_data['id'], 
                              'NObeyesdad': label_encoder.inverse_transform(test_predictions)})

In [ ]:
# Save submission dataframe to CSV
submission_df.to_csv('submission.csv', index=False)

# save the trained model

In [26]:
# Save the best model with pickle
import pickle
with open('trained_models/trained_models.pkl', 'wb') as file:
    pickle.dump(best_model, file)

with open('trained_models/label_encoders.pkl', 'wb') as file:
    pickle.dump(label_encoder, file)
    